# OpenAI埋め込みのベクトルデータベースとしてMyScaleを使用する

このノートブックでは、OpenAI埋め込みのベクトルデータベースとしてMyScaleを使用するためのステップバイステップガイドを提供します。プロセスには以下が含まれます：

1. OpenAI APIによって生成された事前計算済み埋め込みの利用
2. これらの埋め込みをMyScaleのクラウドインスタンスに保存
3. OpenAI APIを使用した生テキストクエリの埋め込みへの変換
4. MyScaleを活用した作成されたコレクション内での最近傍探索の実行

### MyScaleとは

[MyScale](https://myscale.com)は、ベクトル検索とSQL分析を組み合わせて、高性能で合理化された完全管理型の体験を提供するClickhouseベースのデータベースです。構造化データとベクトルデータの両方に対する結合クエリと分析を促進するように設計されており、すべてのデータ処理に対して包括的なSQLサポートを提供します。

### デプロイメントオプション

- [MyScale Console](https://console.myscale.com)を使用して、2分以内にクラスター上でベクトル検索をSQLと共にデプロイし、実行できます。

## 前提条件

このガイドに従うには、以下が必要です：

1. [クイックスタートガイド](https://docs.myscale.com/en/quickstart/)に従ってデプロイされたMyScaleクラスター
2. MyScaleとやり取りするための'clickhouse-connect'ライブラリ
3. クエリのベクトル化のための[OpenAI API key](https://beta.openai.com/account/api-keys)

### 必要な要件のインストール

このノートブックには`openai`、`clickhouse-connect`、およびその他の依存関係が必要です。以下のコマンドを使用してインストールしてください：

In [ ]:
! pip install openai clickhouse-connect wget pandas

### OpenAI APIキーの準備

OpenAI APIを使用するには、APIキーを設定する必要があります。まだお持ちでない場合は、[OpenAI](https://platform.openai.com/account/api-keys)から取得できます。

In [ ]:
import openai

# get API key from on OpenAI website
openai.api_key = "OPENAI_API_KEY"

# check we have authenticated
openai.Engine.list()

## MyScaleに接続する

[接続詳細](https://docs.myscale.com/en/cluster-management/)セクションに従って、MyScaleコンソールからクラスターホスト、ユーザー名、パスワード情報を取得し、以下に示すようにクラスターへの接続を作成してください：

In [1]:
import clickhouse_connect

# initialize client
client = clickhouse_connect.get_client(host='YOUR_CLUSTER_HOST', port=8443, username='YOUR_USERNAME', password='YOUR_CLUSTER_PASSWORD')

## データの読み込み

OpenAIが提供するWikipedia記事の事前計算されたベクトル埋め込みのデータセットを読み込む必要があります。`wget`パッケージを使用してデータセットをダウンロードしてください。

In [ ]:
import wget

embeddings_url = "https://cdn.openai.com/API/examples/data/vector_database_wikipedia_articles_embedded.zip"

# The file is ~700 MB so this will take some time
wget.download(embeddings_url)

ダウンロードが完了したら、`zipfile`パッケージを使用してファイルを展開します：

In [ ]:
import zipfile

with zipfile.ZipFile("vector_database_wikipedia_articles_embedded.zip", "r") as zip_ref:
    zip_ref.extractall("../data")

次に、`vector_database_wikipedia_articles_embedded.csv`からPandas DataFrameにデータを読み込むことができます：

In [ ]:
import pandas as pd

from ast import literal_eval

# read data from csv
article_df = pd.read_csv('../data/vector_database_wikipedia_articles_embedded.csv')
article_df = article_df[['id', 'url', 'title', 'text', 'content_vector']]

# read vectors from strings back into a list
article_df["content_vector"] = article_df.content_vector.apply(literal_eval)
article_df.head()

## インデックスデータ

MyScaleに埋め込みデータを格納するための`articles`というSQLテーブルを作成します。このテーブルには、コサイン距離メトリックを使用したベクトルインデックスと、埋め込みの長さに対する制約が含まれます。以下のコードを使用してarticlesテーブルを作成し、データを挿入してください：

In [ ]:
# create articles table with vector index
embedding_len=len(article_df['content_vector'][0]) # 1536

client.command(f"""
CREATE TABLE IF NOT EXISTS default.articles
(
    id UInt64,
    url String,
    title String,
    text String,
    content_vector Array(Float32),
    CONSTRAINT cons_vector_len CHECK length(content_vector) = {embedding_len},
    VECTOR INDEX article_content_index content_vector TYPE HNSWFLAT('metric_type=Cosine')
)
ENGINE = MergeTree ORDER BY id
""")

# insert data into the table in batches
from tqdm.auto import tqdm

batch_size = 100
total_records = len(article_df)

# upload data in batches
data = article_df.to_records(index=False).tolist()
column_names = article_df.columns.tolist() 

for i in tqdm(range(0, total_records, batch_size)):
    i_end = min(i + batch_size, total_records)
    client.insert("default.articles", data[i:i_end], column_names=column_names)

ベクターインデックスはバックグラウンドで自動的に構築されるため、検索を実行する前にビルドステータスを確認する必要があります。

In [2]:
# check count of inserted data
print(f"articles count: {client.command('SELECT count(*) FROM default.articles')}")

# check the status of the vector index, make sure vector index is ready with 'Built' status
get_index_status="SELECT status FROM system.vector_indices WHERE name='article_content_index'"
print(f"index build status: {client.command(get_index_status)}")

articles count: 25000
index build status: Built


## データの検索

MyScaleにインデックス化された後、ベクトル検索を実行して類似のコンテンツを見つけることができます。まず、OpenAI APIを使用してクエリの埋め込みを生成します。次に、MyScaleを使用してベクトル検索を実行します。

In [4]:
import openai

query = "Famous battles in Scottish history"

# creates embedding vector from user query
embed = openai.Embedding.create(
    input=query,
    model="text-embedding-3-small",
)["data"][0]["embedding"]

# query the database to find the top K similar content to the given query
top_k = 10
results = client.query(f"""
SELECT id, url, title, distance(content_vector, {embed}) as dist
FROM default.articles
ORDER BY dist
LIMIT {top_k}
""")

# display results
for i, r in enumerate(results.named_results()):
    print(i+1, r['title'])

1 Battle of Bannockburn
2 Wars of Scottish Independence
3 1651
4 First War of Scottish Independence
5 Robert I of Scotland
6 841
7 1716
8 1314
9 1263
10 William Wallace
